# Bibliotecas

In [110]:
import math
from itertools import combinations

# Datos

In [111]:
# --- Data ---
circles = [
    ((9.5, 30.5), 3.5),
    ((11, 30.5), 3.5),
    ((18, 5), 5),
    ((4, 23), 3),
    ((4, 17), 4),
    ((7, 17), 4.5),
    ((12.5,17), 3),
    ((18, 15), 6),
    ((27,19),3),
    ((30,17),3),
    ((4, 11), 3),
    ((12, 9), 2),
    ((16,9),3),    
    ((6, 4),  3),
    ((7.5, 4), 2.5),
    ((18, 24), 5)
        ]

points = [
    (28, 18), 
    (9, 17), 
    (18, 9.5), 
    (18, 20), 
    (11, 5), 
    (14, 30.5)
        ]

# Funciones

## Funciones de Q

In [112]:
def build_Q(circles, points):
    Q = []
    # 1) Input points
    for idx, (x, y) in enumerate(points, start=1):
        Q.append((x, y, "input", {idx}))
    # 2) Left endpoints of circles
    for idx, ((cx, cy), r) in enumerate(circles, start=1):
        Q.append((cx - r, cy, "left", {idx}))
    # 3) Order the list Q by x, then by y
    return order_Q(Q)

In [113]:
def order_Q(Q):
    Q.sort(key=lambda t: (t[0], t[1]))
    return Q

In [114]:
def event_in_Q(Q, event, tol=1e-9):
    """
    Regresa True si existe en Q un evento con mismo tipo e ids y con (x,y)
    a distancia <= tol. De lo contrario False.
    event debe ser (x, y, tipo, ids_set)
    """
    x, y, etype, eids = event
    for (qx, qy, qtype, qids) in Q:
        if qtype != etype:
            continue
        if qids != eids:
            continue
        if abs(qx - x) <= tol and abs(qy - y) <= tol:
            return True
    return False


def add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True):
    """
    Agrega a Q los eventos de `nuevos_eventos` (tuplas estilo Q: (x, y, "intersection", {i,j}))
    que no existan ya en Q (según `event_in_Q`).
    Modifica Q in place. Regresa el número de eventos agregados.
    """
    agregados = 0
    for ev in nuevos_eventos:
        # Validación básica de estructura
        if not (isinstance(ev, (list, tuple)) and len(ev) == 4 and isinstance(ev[3], set)):
            continue
        if not event_in_Q(Q, ev, tol=tol):
            Q.append(ev)
            agregados += 1

    # if agregados and keep_sorted:
    #     # Ordena por (x, y) como tu order_Q
    #     Q.sort(key=lambda t: (t[0], t[1]))
    return agregados


In [115]:
Q = build_Q(circles, points)
print(Q)

[(0, 17, 'left', {5}), (1, 11, 'left', {11}), (1, 23, 'left', {4}), (2.5, 17, 'left', {6}), (3, 4, 'left', {14}), (5.0, 4, 'left', {15}), (6.0, 30.5, 'left', {1}), (7.5, 30.5, 'left', {2}), (9, 17, 'input', {2}), (9.5, 17, 'left', {7}), (10, 9, 'left', {12}), (11, 5, 'input', {5}), (12, 15, 'left', {8}), (13, 5, 'left', {3}), (13, 9, 'left', {13}), (13, 24, 'left', {16}), (14, 30.5, 'input', {6}), (18, 9.5, 'input', {3}), (18, 20, 'input', {4}), (24, 19, 'left', {9}), (27, 17, 'left', {10}), (28, 18, 'input', {1})]


## Funciones de L

In [116]:
eps = 1e-9
x0 = Q[0][0] - eps

sweepL = {
    "x": x0,
    "active": set(),
    "conditions": [
        {"in": set(), "out": set(), "ineq": []}
    ]
}

In [117]:
def advance_sweep_to(sweepL, x_event, circles):
    sweepL["x"] = x_event

## Condiciones

In [118]:
def in_circle(x, y, cx, cy, r, tolerancia=1e-12):
    """
    Evalua si un punto se encuentra dentro de un círculo.
    Regresa Verdadero o Falso, dependiendo si cumpe o no.
    Se usa una tolerancia para evitar errores por punto flotante
    """
    print("Evalua el punto (",x,",",y,") en el círculo con centro en (",cx,",",cy,") y radio ",r)
    print((x - cx)**2 + (y - cy)**2, " <= ", r**2 + tolerancia, (x - cx)**2 + (y - cy)**2 <= r**2 + tolerancia)
    return (x - cx)**2 + (y - cy)**2 <= r**2 + tolerancia

In [119]:
def circle_y_bounds_at_x(circle, x0):
    """
    Regresa los cortes (y_low, y_high) de la línea x = x0 
    con el círculo con centro en (cx, cy) y radio r
    Si la línea no intersecta al círculo regresa (None, None).
    """
    (cx, cy), r = circle
    # Calcula la distancia horizontal entre el centro del círculo
    # y el punto x0
    dx = x0 - cx
    # Si esa distancia es mayor al radio, no se intersectan
    if abs(dx) > r:
        return None, None
    # Si s[i se intersectan, calcula las intersecciones
    h = math.sqrt(max(r*r - dx*dx, 0.0))
    return cy - h, cy + h

In [120]:
# Hay 2 tipos de condiciones que puede tener una región
# con respecto a un cículo activo en la línea de barrido sweepL:
#    * Que esté dentro de ese círculo activo en sweepL
#    * Que esté fuera de ese círculo activo en sweepL
# y, si está fuera puede estar:
#    * Por encima de ese círculo
#    * Por debajo de ese círculo

def satisfies(cond, x, y, circles):
    # para evaluar si está dentro del círculo
    for cid in cond.get("in", set()):
        (cx, cy), r = circles[cid - 1]
        if not in_circle(x, y, cx, cy, r):
            return False

    # para evaluar si está fuera del circulo 
    for cid in cond.get("out", set()):
        (cx, cy), r = circles[cid - 1]
        if in_circle(x, y, cx, cy, r):
            return False

    # para evaluar si está por encima o por debajo
    for var, op, (cid, which) in cond.get("ineq", []):
        y_low, y_high = circle_y_bounds_at_x(circles[cid - 1], x)
        if y_low is None:
            # If the sweep line x doesn't intersect this circle at all,
            # this inequality is not meaningful. Be conservative and fail.
            return False
        y_ref = y_low if which == "y_low" else y_high
        if op == "<" and not (y < y_ref): return False
        if op == ">" and not (y > y_ref): return False

    return True

## Leftend point

In [121]:
def leftend_point(Q, sweepL, circle_id, circles, eps=1e-6):
    """
    Insert circle `circle_id` into the conditions model at sweepL['x'].
    We only add the logical conditions; boundaries are computed lazily in `satisfies`.
    """
    (cx, cy), r = circles[circle_id - 1]
    print("Círculo izquierdo con centro en (",cx,",",cy,") y radio ",r)
    
    # Evaluate just to the right of the left endpoint to avoid tangency
    x_prime = sweepL["x"] + eps
    dx = x_prime - cx
    if abs(dx) > r + 1e-15:
        # numerically outside the circle's vertical span at x', nothing to split
        sweepL.setdefault("active", set()).add(circle_id)
        return Q, sweepL

    # Find the single condition that contains (x_prime, cy) *before* inserting the circle
    conds = sweepL["conditions"]
    print("Conditions",conds)
    k = None
    for i, cond in enumerate(conds):
        if satisfies(cond, x_prime, cy, circles):
            k = i
            break
    if k is None:
        k = len(conds) - 1  # fallback

    base = conds[k]
    print("base", base)
    base_in   = set(base.get("in",  set()))
    base_out  = set(base.get("out", set()))
    base_ineq = list(base.get("ineq", []))

    # Replace with three regions using symbolic inequalities tied to (circle_id, "y_low"/"y_high")
    down = {
        "in":   set(base_in),
        "out":  set(base_out) | {circle_id},
        "ineq": base_ineq + [("y", "<", (circle_id, "y_low"))],
    }
    middle = {
        "in":   set(base_in) | {circle_id},
        "out":  set(base_out),
        "ineq": list(base_ineq),
    }
    up = {
        "in":   set(base_in),
        "out":  set(base_out) | {circle_id},
        "ineq": base_ineq + [("y", ">", (circle_id, "y_high"))],
    }

    print("down", down)
    print("middle", middle)
    print("up", up)

    print("Revisar intersección arriba: ", mark_intersection_needed(up))
    if mark_intersection_needed(up):
        print(intersections_from_condition(up, circles))
        nuevos_eventos = intersections_from_condition(up, circles)
        add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True)
    print("Revisar intersección abajo: ", mark_intersection_needed(down))
    if mark_intersection_needed(down):
        print(intersections_from_condition(down, circles))
        nuevos_eventos = intersections_from_condition(down, circles)
        add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True)

    Q.append((cx + r, cy, "right", {circle_id}))
    order_Q(Q)
    print("Q: ",Q)

    sweepL["conditions"] = conds[:k] + [down, middle, up] + conds[k+1:]
    sweepL.setdefault("active", set()).add(circle_id)

    print("L", sweepL)
    return Q, sweepL

In [122]:
def mark_intersection_needed(cond, var="y"):
    """
    Return 'check for intersection' if `cond["ineq"]` contains BOTH '<' and '>' 
    inequalities for the given variable (default 'y'). Supports ANY number of inequalities.
    Otherwise return None.
    """
    lt = gt = False
    for item in cond.get("ineq", []):
        if not isinstance(item, (list, tuple)) or len(item) != 3:
            continue
        v, op, _ = item
        if v != var:
            continue
        if op == "<":
            lt = True
        elif op == ">":
            gt = True
        if lt and gt:
            return True
    return None


# Example:
down = {'in': {11}, 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '<', (11, 'y_low'))]}
up   = {'in': {11}, 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '>', (11, 'y_high'))]}
print(mark_intersection_needed(up))  # -> "check for intersection"
print(mark_intersection_needed(down))

True
None


In [123]:
def circle_circle_intersections(c1, c2, tol=1e-12):
    (x0, y0), r0 = c1
    (x1, y1), r1 = c2
    dx, dy = x1 - x0, y1 - y0
    d = math.hypot(dx, dy)

    # no solutions or infinite solutions (coincident) -> return none
    if d > r0 + r1 + tol:         # separate
        return []
    if d < abs(r0 - r1) - tol:    # contained
        return []
    if d < tol and abs(r0 - r1) < tol:  # coincident
        return []

    # base point along the line of centers
    a = (r0*r0 - r1*r1 + d*d) / (2*d)
    h2 = r0*r0 - a*a
    if h2 < -tol:
        return []
    h = math.sqrt(max(h2, 0.0))

    xm = x0 + a * dx / d
    ym = y0 + a * dy / d

    if h <= tol:  # tangent (one point)
        return [(xm, ym)]

    # two intersection points
    rx = -dy * (h / d)
    ry =  dx * (h / d)
    return [(xm + rx, ym + ry), (xm - rx, ym - ry)]

def intersections_from_condition(cond, circles, var="y"):
    """
    Collect circle IDs from cond['ineq'] (for `var`), compute pairwise intersections,
    and return Q-style tuples: (x, y, "intersection", {cid1, cid2}).
    """
    circle_ids = sorted({cid for v, _, (cid, _) in cond.get("ineq", []) if v == var})
    if len(circle_ids) < 2:
        return []

    events = []
    for i, j in combinations(circle_ids, 2):
        pts = circle_circle_intersections(circles[i - 1], circles[j - 1])
        for (x, y) in pts:
            events.append((x, y, "intersection", {i, j}))
    return events

# Example:
up = {'in': {11}, 'out': {11, 5}, 'ineq': [('y','<',(5,'y_low')), ('y','>',(11,'y_high'))]}
print(intersections_from_condition(up, circles))


[(5.777560750641795, 13.416666666666666, 'intersection', {11, 5}), (2.2224392493582052, 13.416666666666666, 'intersection', {11, 5})]


In [125]:
# split_segment_for_new_circle(sweepL, circle_id=5, circles=circles, eps=1e-6)
leftend_point(Q, sweepL, circle_id=5, circles=circles, eps=1e-6)

Círculo izquierdo con centro en ( 4 , 17 ) y radio  4
Conditions [{'in': set(), 'out': set(), 'ineq': []}]
base {'in': set(), 'out': set(), 'ineq': []}
down {'in': set(), 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]}
middle {'in': {5}, 'out': set(), 'ineq': []}
up {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}
Revisar intersección arriba:  None
Revisar intersección abajo:  None
Q:  [(0, 17, 'left', {5}), (1, 11, 'left', {11}), (1, 23, 'left', {4}), (2.5, 17, 'left', {6}), (3, 4, 'left', {14}), (5.0, 4, 'left', {15}), (6.0, 30.5, 'left', {1}), (7.5, 30.5, 'left', {2}), (8, 17, 'right', {5}), (9, 17, 'input', {2}), (9.5, 17, 'left', {7}), (10, 9, 'left', {12}), (11, 5, 'input', {5}), (12, 15, 'left', {8}), (13, 5, 'left', {3}), (13, 9, 'left', {13}), (13, 24, 'left', {16}), (14, 30.5, 'input', {6}), (18, 9.5, 'input', {3}), (18, 20, 'input', {4}), (24, 19, 'left', {9}), (27, 17, 'left', {10}), (28, 18, 'input', {1})]
L {'x': -1e-09, 'active': {5}, 'conditions': [{'in': 

([(0, 17, 'left', {5}),
  (1, 11, 'left', {11}),
  (1, 23, 'left', {4}),
  (2.5, 17, 'left', {6}),
  (3, 4, 'left', {14}),
  (5.0, 4, 'left', {15}),
  (6.0, 30.5, 'left', {1}),
  (7.5, 30.5, 'left', {2}),
  (8, 17, 'right', {5}),
  (9, 17, 'input', {2}),
  (9.5, 17, 'left', {7}),
  (10, 9, 'left', {12}),
  (11, 5, 'input', {5}),
  (12, 15, 'left', {8}),
  (13, 5, 'left', {3}),
  (13, 9, 'left', {13}),
  (13, 24, 'left', {16}),
  (14, 30.5, 'input', {6}),
  (18, 9.5, 'input', {3}),
  (18, 20, 'input', {4}),
  (24, 19, 'left', {9}),
  (27, 17, 'left', {10}),
  (28, 18, 'input', {1})],
 {'x': -1e-09,
  'active': {5},
  'conditions': [{'in': set(), 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]},
   {'in': {5}, 'out': set(), 'ineq': []},
   {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}]})

In [126]:
# Advance to circle 11's left endpoint
(cx, cy), r = circles[11 - 1]
advance_sweep_to(sweepL, cx - r, circles)

leftend_point(Q, sweepL, circle_id=11, circles=circles, eps=1e-6)

Círculo izquierdo con centro en ( 4 , 11 ) y radio  3
Conditions [{'in': set(), 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]}, {'in': {5}, 'out': set(), 'ineq': []}, {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}]
Evalua el punto ( 1.000001 , 11 ) en el círculo con centro en ( 4 , 17 ) y radio  4
44.999994000000996  <=  16.000000000001 False
base {'in': set(), 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]}
down {'in': set(), 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '<', (11, 'y_low'))]}
middle {'in': {11}, 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]}
up {'in': set(), 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '>', (11, 'y_high'))]}
Revisar intersección arriba:  True
[(5.777560750641795, 13.416666666666666, 'intersection', {11, 5}), (2.2224392493582052, 13.416666666666666, 'intersection', {11, 5})]
Revisar intersección abajo:  None
Q:  [(0, 17, 'left', {5}), (1, 11, 'left', {11}), (1, 23, 'left', {4}), (2.2224392493582052, 13.416666666

([(0, 17, 'left', {5}),
  (1, 11, 'left', {11}),
  (1, 23, 'left', {4}),
  (2.2224392493582052, 13.416666666666666, 'intersection', {5, 11}),
  (2.5, 17, 'left', {6}),
  (3, 4, 'left', {14}),
  (5.0, 4, 'left', {15}),
  (5.777560750641795, 13.416666666666666, 'intersection', {5, 11}),
  (6.0, 30.5, 'left', {1}),
  (7, 11, 'right', {11}),
  (7.5, 30.5, 'left', {2}),
  (8, 17, 'right', {5}),
  (9, 17, 'input', {2}),
  (9.5, 17, 'left', {7}),
  (10, 9, 'left', {12}),
  (11, 5, 'input', {5}),
  (12, 15, 'left', {8}),
  (13, 5, 'left', {3}),
  (13, 9, 'left', {13}),
  (13, 24, 'left', {16}),
  (14, 30.5, 'input', {6}),
  (18, 9.5, 'input', {3}),
  (18, 20, 'input', {4}),
  (24, 19, 'left', {9}),
  (27, 17, 'left', {10}),
  (28, 18, 'input', {1})],
 {'x': 1,
  'active': {5, 11},
  'conditions': [{'in': set(),
    'out': {5, 11},
    'ineq': [('y', '<', (5, 'y_low')), ('y', '<', (11, 'y_low'))]},
   {'in': {11}, 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]},
   {'in': set(),
    'out': {5, 

In [127]:
sweepL

{'x': 1,
 'active': {5, 11},
 'conditions': [{'in': set(),
   'out': {5, 11},
   'ineq': [('y', '<', (5, 'y_low')), ('y', '<', (11, 'y_low'))]},
  {'in': {11}, 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]},
  {'in': set(),
   'out': {5, 11},
   'ineq': [('y', '<', (5, 'y_low')), ('y', '>', (11, 'y_high'))]},
  {'in': {5}, 'out': set(), 'ineq': []},
  {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}]}

## Rightendpoint

In [130]:
def rightend_point(Q, sweepL, circle_id, circles, eps=1e-6):
    """
    Procesa el extremo derecho del círculo `circle_id` en la posición sweepL['x'].
    - Busca la región 'middle' (condición con circle_id en 'in') que contiene (x - eps, cy)
    - Toma sus vecinas 'down' (k-1) y 'up' (k+1)
    - Opcional: detecta intersecciones en up/down (igual que en leftend_point)
    - Elimina la región 'middle' y fusiona up y down removiendo referencias al círculo
    - No agrega evento 'right' a Q
    """
    (cx, cy), r = circles[circle_id - 1]
    print("Círculo derecho con centro en (", cx, ",", cy, ") y radio ", r)

    x_prime = sweepL["x"] - eps  # evaluar justo antes del extremo derecho
    conds = sweepL["conditions"]
    print("Conditions", conds)

    # 1) localizar la región 'middle' (con circle_id en 'in') que contiene (x', cy)
    k = None
    for i, cond in enumerate(conds):
        if circle_id in cond.get("in", set()) and satisfies(cond, x_prime, cy, circles):
            k = i
            break

    if k is None:
        # No se encontró región media; desactivar y salir
        sweepL.setdefault("active", set()).discard(circle_id)
        print("No se encontró región con el círculo en 'in' para (x', cy); no se fusiona.")
        return Q, sweepL

    # Deben existir vecinos arriba y abajo
    if k - 1 < 0 or k + 1 >= len(conds):
        sweepL.setdefault("active", set()).discard(circle_id)
        print("No hay vecinos up/down contiguos; no se fusiona.")
        return Q, sweepL

    down  = conds[k - 1]
    mid   = conds[k]
    up    = conds[k + 1]

    print("down", down)
    print("middle", mid)
    print("up", up)

    # 2) Igual que en leftend_point: revisar intersecciones en up y down
    if mark_intersection_needed(up):
        print("Revisar intersección arriba:", intersections_from_condition(up, circles))
        nuevos_eventos = intersections_from_condition(up, circles)
        add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True)

    if mark_intersection_needed(down):
        print("Revisar intersección abajo:", intersections_from_condition(down, circles))
        nuevos_eventos = intersections_from_condition(down, circles)
        add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True)

    # 3) Fusionar up y down en una región base, quitando referencias a circle_id
    def _sin_ineq_del_circulo(ineqs, cid):
        # elimina desigualdades que referencien a este círculo
        filtradas = []
        for t in ineqs:
            if not (isinstance(t, (list, tuple)) and len(t) == 3):
                filtradas.append(t); continue
            var, op, ref = t
            if var == "y" and isinstance(ref, tuple) and ref[0] == cid:
                continue  # quitarla
            filtradas.append(t)
        return filtradas

    # in/out base: intersección de ambos vecinos (y quitando el círculo de 'out')
    base_in  = set(down.get("in", set())) & set(up.get("in", set()))
    base_out = (set(down.get("out", set())) & set(up.get("out", set()))) - {circle_id}

    # ineq base: unión de ineqs de up y down sin referencias al círculo que cierra (deduplicada)
    ineq_down = _sin_ineq_del_circulo(down.get("ineq", []), circle_id)
    ineq_up   = _sin_ineq_del_circulo(up.get("ineq", []), circle_id)

    base_ineq = []
    for t in ineq_down + ineq_up:
        if t not in base_ineq:
            base_ineq.append(t)

    merged = {"in": base_in, "out": base_out, "ineq": base_ineq}
    print("merged", merged)

    # 4) Reemplazar [down, middle, up] por [merged]
    sweepL["conditions"] = conds[:k - 1] + [merged] + conds[k + 2:]
    sweepL.setdefault("active", set()).discard(circle_id)

    print("L (tras cerrar círculo)", sweepL)
    # Nota: NO agregamos evento 'right' a Q
    return Q, sweepL


In [132]:
# Avanza a la coordenada del extremo derecho del círculo 11
(cx, cy), r = circles[11 - 1]
advance_sweep_to(sweepL, cx + r, circles)

# Procesa el extremo derecho del círculo 11
Q, sweepL = rightend_point(Q, sweepL, circle_id=11, circles=circles, eps=1e-6)

# Opcional: inspeccionar estado
print("Q:", Q)
print("L:", sweepL)


Círculo derecho con centro en ( 4 , 11 ) y radio  3
Conditions [{'in': set(), 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '<', (11, 'y_low'))]}, {'in': {11}, 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]}, {'in': set(), 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '>', (11, 'y_high'))]}, {'in': {5}, 'out': set(), 'ineq': []}, {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}]
Evalua el punto ( 6.999999 , 11 ) en el círculo con centro en ( 4 , 11 ) y radio  3
8.999994000001  <=  9.000000000001 True
Evalua el punto ( 6.999999 , 11 ) en el círculo con centro en ( 4 , 17 ) y radio  4
44.999994000000996  <=  16.000000000001 False
down {'in': set(), 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '<', (11, 'y_low'))]}
middle {'in': {11}, 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]}
up {'in': set(), 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '>', (11, 'y_high'))]}
Revisar intersección arriba: [(5.777560750641795, 13.416666666666

# Algoritmo final

In [131]:
def algoritmo(circles, points):
    # Inicializar variables
    Ac = circles
    Q = build_Q(circles, points)
    eps = 1e-9
    x0 = Q[0][0] - eps
    sweepL = {
    "x": x0,
    "active": set(),
    "conditions": [
        {"in": set(), "out": set(), "ineq": []}
        ]
    }
    print("Antes de iniciar:")
    print("Q = ", Q)
    print("Línea de barrido =", sweepL)
    print("Ac =", Ac)
    i = 0
    while i < len(Q):
        x, y, kind, idx = Q[i]

        advance_sweep_to(sweepL, x, circles)
        
        if kind == "input":
            print(f"Point ({x}, {y}) is an INPUT point, index {idx}")
    
        elif kind == "left":
            circle_idx = list(idx)[0]  # get the circle index from the set
            print(f"Point ({x}, {y}) is a LEFT endpoint of circle {idx}")
            Q, sweepL = leftend_point(Q, sweepL, circle_id=circle_idx, circles=circles, eps=1e-6)
    
        elif kind == "right":
            circle_idx = list(idx)[0]
            print(f"Point ({x}, {y}) is a RIGHT endpoint of circle {idx}")
            Q, sweepL = rightend_point(Q, sweepL, circle_id=circle_idx, circles=circles, eps=1e-6)

        elif kind == "intersection":
            print(f"Point ({x}, {y}) is a INTERSECTION point of circles {idx}")
        i += 1




algoritmo(circles, points)

Antes de iniciar:
Q =  [(0, 17, 'left', {5}), (1, 11, 'left', {11}), (1, 23, 'left', {4}), (2.5, 17, 'left', {6}), (3, 4, 'left', {14}), (5.0, 4, 'left', {15}), (6.0, 30.5, 'left', {1}), (7.5, 30.5, 'left', {2}), (9, 17, 'input', {2}), (9.5, 17, 'left', {7}), (10, 9, 'left', {12}), (11, 5, 'input', {5}), (12, 15, 'left', {8}), (13, 5, 'left', {3}), (13, 9, 'left', {13}), (13, 24, 'left', {16}), (14, 30.5, 'input', {6}), (18, 9.5, 'input', {3}), (18, 20, 'input', {4}), (24, 19, 'left', {9}), (27, 17, 'left', {10}), (28, 18, 'input', {1})]
Línea de barrido = {'x': -1e-09, 'active': set(), 'conditions': [{'in': set(), 'out': set(), 'ineq': []}]}
Ac = [((9.5, 30.5), 3.5), ((11, 30.5), 3.5), ((18, 5), 5), ((4, 23), 3), ((4, 17), 4), ((7, 17), 4.5), ((12.5, 17), 3), ((18, 15), 6), ((27, 19), 3), ((30, 17), 3), ((4, 11), 3), ((12, 9), 2), ((16, 9), 3), ((6, 4), 3), ((7.5, 4), 2.5), ((18, 24), 5)]
Point (0, 17) is a LEFT endpoint of circle {5}
Círculo izquierdo con centro en ( 4 , 17 ) y radio